In [1]:
# -*- coding=utf-8 -*-
"""
주요 기능: Google Sheets(API)
    - CRUD

사용 방법: 
    - 

참조 자료:
    - https://developers.google.com/sheets/api/reference/rest/v4/spreadsheets/request?authuser=0
"""

##@@@ Package/Module
##============================================================

##@@ Built-In Package/Module
##------------------------------------------------------------
import os, sys
import json
import ast

##@@ External Package/Module
##------------------------------------------------------------
import pandas as pd

## Local(PC Drive) 환경인지 Cloud(Google Drive) 환경인지 확인
ENV = "_LOCAL_"
if "content" in os.getcwd():
    ENV = "_GDRIVE_"

if ENV == "_LOCAL_":
    try: # LOCAL SHELL
        cwd = __file__
    except:  # LODAL JUPYTER
        import import_ipynb  ## NOTE: .ipynb을 import 할 수 있도록 하는 모듈
else:
    sys.path.insert(0, '/content/drive/MyDrive/__COLAB/PACKAGES')
    import import_ipynb  ## NOTE: .ipynb을 import 할 수 있도록 하는 모듈
    sys.path.insert(0, '/content/drive/MyDrive/__COLAB/__GOOGLE')
    ## NOTE: 현재 디렉토리로 이동한 후 import해야 함
    # %cd /content/drive/MyDrive/__COLAB/__GOOGLE

from GoogleAPI import GoogleAPI
PATH_CONFIGS = "./configs"

##@@@ Constant/Varible
##============================================================

##@@ Constant
##------------------------------------------------------------


##@@ Variable(Golobal)
##------------------------------------------------------------
# _SERVICES = []  ## 서비스 목록 [{'api_name': 'drive', 'user_nick': 'deverlife', 'bot_nick': None, 'service': api_OBJ1}, {'api_name': 'sheets', 'user_nick': None, 'bot_nick': 'mats', 'service': api_OBJ2}, ...] 


##@@@ Private Function
##============================================================

##@@ data
##------------------------------------------------------------
def _data_for_sheet(data=[], columns=[]):
    [list(data[0].keys())] + pd.DataFrame(data).values.tolist()


##@@ column, range
##------------------------------------------------------------
def _colnum_to_colstr(n):
    colstr = ""
    while n > 0:
        n, remainder = divmod(n - 1, 26)
        colstr = chr(65 + remainder) + colstr
    return colstr


def _get_sheet_range(h_w=(1,1), start=(0,0)):
    """_get_sheet_range: 행,열 range -> sheet_range
    Desc:
        - start(행번호, 열번호), h_w(행길이, 열길이) -> sheet_range

    Args:
        - h_w(tuple, (1,1)): h_w(행길이, 열길이)
        - start(tuple, (0,0)): start(행번호, 열번호)

    Returns:
        - str: "A1:B2"

    Usages:
        - [description]
    """
    if h_w == None:
        return None

    row_bgn = start[0] + 1
    row_end = row_bgn + h_w[0]
    col_bgn = start[1] + 1
    col_end = start[1] + h_w[1]

    return f"{_colnum_to_colstr(col_bgn)}{row_bgn}:{_colnum_to_colstr(col_end)}{row_end}"      


def _get_sheet_range2(rowcol_range=[(0,0), (1,1)]):
    """_get_sheet_range: row, col 숫자 -> sheet_range
    Args:
        - rowcol_range(list, [(0,0), (1,1)]): [(시작행번호, 시작열번호), (종료행번호, 종료열번호)]

    Returns:
        - str: "A1:B2"
    """
    if rowcol_range == None:
        return None

    row_bgn = rowcol_range[0][0] + 1
    row_end = rowcol_range[1][0] + 1
    col_bgn = _colnum_to_colstr(rowcol_range[0][1] + 1)
    col_end = _colnum_to_colstr(rowcol_range[1][1] + 1)

    return f"{col_bgn}{row_bgn}:{col_end}{row_end}"


##@@ read
##------------------------------------------------------------
def _eval_read_sheet(data):
    header = data.pop(0)
    dicts = []
    for row in data:
        dicts.append(
            {  ## NOTE: v가 list 이거나 dict 이면 parsing, eval(v)도 가능하나 ast.literal_eval(v)를 권장
                k: ast.literal_eval(v) if (v[0] == "[" and v[-1] == "]") or (v[0] == "{" and v[-1] == "}") else v
                for (k, v) in zip(header, row)
            }
        )
    # with open("test.json", "w") as f:
    #     json.dump(data, f, ensure_ascii=False, indent=4)
    return dicts


##@@ update
##------------------------------------------------------------
# 2. 입력 columns의 header에서의 index 구함
def _sorted_idxs_cols(columns, header):
    """_idxs_cols: 입력 제목행(columns), 기존 제목행(header) -> 기존/신규 행렬,인덱스
    Desc:
        - [extended_summary]

    Args:
        - columns(list): 입력 제목행(update 예정인 제목행)
        - header(list): 기존 제목행(update 대상 Google Sheets에서 가져옴)

    Returns:
        - [tuple]: old(기존에 있는 제목행), new(새로 입력되어지는 제목행)
        * old_idxs / old_cols: 기존 header 순으로 sort

    Usages:
        - [description]
    """
    l = len(header)
    old_idxs = sorted([header.index(col) for col in columns if col in header])
    old_cols = [header[idx] for idx in old_idxs]
    new_cols = [col for col in columns if not col in header]
    new_idxs = [i + l for i in range(len(new_cols))]

    return (old_idxs, old_cols, new_idxs, new_cols)

# 4. header index의 연속/불연속 구간 index 구함
def _continuous_idxs_cols(old_idxs, old_cols, new_idxs, new_cols, header):
    ## cont_idxs
    cont_idxs = [[-1]]
    for idx in old_idxs:
        if idx > cont_idxs[-1][-1] + 1: # 불연속 시작
            # conts[-1][1] = idx
            cont_idxs.append([idx])
        else:  # 연속
            cont_idxs[-1].append(idx)

    cont_idxs.pop(0)

    ## cont_cols
    cont_cols = []
    for idxs in cont_idxs:
        cont_cols.append([])
        for idx in idxs:
            cont_cols[-1].append(header[idx])

    ## 신규 항목(제목행) 추가
    cont_idxs.append(new_idxs)
    cont_cols.append(new_cols)
    
    return (cont_idxs, cont_cols)


## column 변경 순서/불연속 적용 dataframe 만듬
def _data_sheet_values(data, columns, cont_cols):
    df = pd.DataFrame(data=data, columns=columns)
    sheet_values = []
    for partial in cont_cols:
        print(df[partial].values.tolist())
        sheet_values.append(df[partial].values.tolist())
    return sheet_values


##@@@ Main Class
##============================================================

class GoogleSheets(GoogleAPI):
    def __init__(self, user_nick=None, bot_nick=None, path=None):
        """__init__: Object 초기화
        Desc:
            - Google Drive Service 연결(user_nick)
            - path 폴더 기준

        Args:
            - user_nick(str, None): 구글계정 별칭(master: monblue@snu.ac.kr / moonitdev: moonitdev@gmail.com / ...)
            - bot_nick(str, None): GCP(google cloud platform) Project 서비스 계정(Bot) 별칭(mats, moonMaster, ...)
            - path(str, None): Google Drive 기준 폴더(경로) 예) __COLAB/_TEST

        Usages:
            - [description]
        """
        super().__init__("sheets", user_nick=user_nick, bot_nick=bot_nick)
        # self.service = build("drive", self.version["drive"], credentials = self.credentials)
        # self._set_id(path)


    ##@@ Private Function
    ##------------------------------------------------------------
    # def _set_id(self, name):  ## TODO: 해당 api에 속하는 파일에서만 검색하도록
    #     id = self._id_by_name(name, self.api_name)
    #     if id == None:
    #         self.create_book(name)
    #     else:
    #         self.id = id
    #         self.name = name

    def _set_id(self, path, auto_create=True):
        self.path = path
        # self.id = self._root_folder_id()  # self.id 초기화
        paths = self._ids_by_path(path=path, folder=False)  # GoogleAPI에서 상속
        id = list(paths[-1].values())[0]  # 마지막 path 항목 id
        if auto_create:  # 폴더/spreadsheets 자동 생성
            id = self._create_folders_by_paths(paths)

        self.id = id if id != None else self.id


    def _start_end_header(self, sheet_name, gap=0):
        """_start_end_header: sheet의 해더, 시작셀, 종료셀
        Desc:
            - [extended_summary]

        Args:
            - sheet_name(str): [description]
            - gap(int, 0): 비어있지 않은 첫번째행과 해더행의 격차

        Returns:
            - [type]: [description]

        Usages:
            - [description]
        """
        rows = self.read_sheet(sheet_name)
        # print(rows)
        
        for i, row in enumerate(rows):
            if row != []:
                start_row = i
                break
        
        for j, col in enumerate(rows[start_row+gap]):  ## NOTE: header행 기준
            if col != '':
                start_col = j
                break
        
        end_row = start_row + len(rows) - 1
        end_col = len(rows[start_row+gap]) - 1

        return [(start_row, start_col), (end_row, end_col), rows[start_row+gap][start_col:]]  ## [시작셀, 종료셀, 해더]


    def _sheet_id(self, sheet_name):
        for sheet in self.book_obj()['sheets']:
            if sheet['properties']['title'] == sheet_name:
                return sheet['properties']['sheetId']


    ##@@ Public Function
    ##------------------------------------------------------------
    def list_of_book(self):
        return self._list_of_file("sheets")


    def change_book(self, name):
        self._set_id(name)


    def save_list_of_book(self, path):
        data = self.list_of_book()
        if data != None:
            pass
            # _write_file(data=data, path=path)


    def book_obj(self):
        return self.service.spreadsheets().get(spreadsheetId=self.id).execute()


    def sheet_obj(self, sheet_name=""):
        return self.service.spreadsheets().values().get(
            spreadsheetId = self.id,
            # majorDimension = 'ROWS',
            range = f"{sheet_name}"
        ).execute()


    def list_of_sheets(self):
        book_obj = self.book_obj()
        return [sheet['properties']['title'] for sheet in book_obj['sheets']]


    # https://developers.google.com/sheets/api/reference/rest/v4/spreadsheets/create
    def create_book(self, name, folder="TEST"):  ## TODO: folder 지정, 동일 이름이 존재하는 경우에 이름 변경
        self.name = name
        self.id = self.service.spreadsheets().create(
            body = {
                'properties': {
                    'title': name
                }
            },
            fields = 'spreadsheetId'
        ).execute().get('spreadsheetId')

        print(f"<{name}> spreadsheet를 생성합니다.")
        
        return self.id


    def create_book(self, name, folder="TEST"):  ## TODO: folder 지정, 동일 이름이 존재하는 경우에 이름 변경
        self.name = name
        self.id = self.service.spreadsheets().create(
            body = {
                'properties': {
                    'title': name
                }
            },
            fields = 'spreadsheetId'
        ).execute().get('spreadsheetId')

        print(f"<{name}> spreadsheet를 생성합니다.")
        
        return self.id


    def _add_sheet(self, sheet_name):
        """add_sheet_by_id: [summary]
        Desc:
            - [extended_summary]

        Args:
            - self.id([type]): [description]
            - sheet_name([type]): [description]

        Returns:
            - [type]: [description]

        Usages:
            - [description]
        
        Refs:
            - https://developers.google.com/sheets/api/samples/sheet
        """       
        return self.service.spreadsheets().batchUpdate(
            spreadsheetId = self.id,
            body = {
                "requests": [
                    {
                        "addSheet": {
                            "properties": {
                                "title": sheet_name
                            }
                        }
                    }
                ]
            }
        ).execute()


    def add_sheet(self, sheet_name):  ## TODO: folder 지정
        if not sheet_name in self.list_of_sheets():  ## NOTE: sheet가 없으면 생성
            self._add_sheet(sheet_name=sheet_name)
        else:
            pass
            # print("{sheet_name} sheet가 존재합니다")


    # https://developers.google.com/sheets/api/guides/values#reading
    def read_sheet(self, sheet_name="API", sheet_range=None):
        # sheet_range = "A2:E"
        range = f"{sheet_name}!{sheet_range}" if sheet_range else f"{sheet_name}"
        
        self.add_sheet(sheet_name) ## NOTE: sheet가 없으면 생성
        try:
            return self.service.spreadsheets().values().get(
                spreadsheetId = self.id,
                # majorDimension = 'ROWS',
                range = range
            ).execute()['values']
        except:  ## NOTE: sheet가 비어있거나, 에러가 발생하면
            return None


    ## https://developers.google.com/sheets/api/guides/values
    def write_sheet(self, sheet_name="Sheet1", sheet_range=None, values=[]):
        self.add_sheet(sheet_name) ## NOTE: sheet가 없으면 생성

        # (sheet_range, input_values) = _data_for_sheet(data=data, columns=columns)
        range = f"{sheet_name}!{sheet_range}" if sheet_range != None else f"{sheet_name}"
        body = {
            "valueInputOption": "RAW", 
            "data": [{
                "range": range,
                "majorDimension": "ROWS",
                "values": values
            }]
        }
        return self.service.spreadsheets().values().batchUpdate(
            spreadsheetId = self.id, 
            body = body
        ).execute()


    def append_sheet(self, sheet_name="Sheet1", sheet_range=None, values=[]):
        range = f"{sheet_name}!{sheet_range}" if sheet_range != None else f"{sheet_name}"
        return self.service.spreadsheets().values().append(
            spreadsheetId = self.id, 
            range = range,
            valueInputOption = "RAW", 
            body = {
                'values': values
            }
        ).execute()


    # def update_sheet(self, sheet_name, columns, data, mode="w"):  ## TODO: mode: "w(rite)" -> 덮어쓰기, "a(ppend)" -> 이어쓰기, "n(ew)" -> 새로쓰기(기존 내용을 전부 지움)
    def update_sheet(self, sheet_name, columns, data):
        (start, end, header) = gss._start_end_header(sheet_name)  ## start, header 정보를 가져옴

        (old_idxs, old_cols, new_idxs, new_cols) = _sorted_idxs_cols(columns, header)
        (cont_idxs, cont_cols) = _continuous_idxs_cols(old_idxs, old_cols, new_idxs, new_cols, header)

        ## data 내용
        sheet_values = _data_sheet_values(data, columns, cont_cols)
        sheet_ranges = []
        for idxs in cont_idxs:
            sheet_range = _get_sheet_range((len(sheet_values[0]), len(idxs)), (start[0]+1, start[1] + idxs[0])) ## NOTE: start=(1,0) 2번째 행, 1번째 열부터 내용 입력(제목행 제외)
            sheet_ranges.append(sheet_range)

        for i, sheet_range in enumerate(sheet_ranges):
            self.write_sheet(sheet_name, sheet_range=sheet_range, values=sheet_values[i])

        ## 추가된 header
        sheet_range = _get_sheet_range(h_w=(1, len(new_idxs)), start=(start[0], start[1] + new_idxs[0]))
        self.write_sheet(sheet_name, sheet_range=sheet_range, values=[new_cols])


    ## https://developers.google.com/sheets/api/reference/rest/v4/spreadsheets/request#findreplacerequest
    def replace_text(self, find, replacement, sheet_name=None):
        if sheet_name == None:
            all_sheets = True
            _range = None
        else:
            all_sheets = None
            _range = {
                "sheetId": self._sheet_id(sheet_name)
            }
        
        requests = []
        requests.append({
            'findReplace': {
                'find': find,
                'replacement': replacement,
                # 'allSheets': False,
                'allSheets': all_sheets,
                "range": _range
                # "range": {
                #     "sheetId": self._sheet_id(sheet_name),  ## NOTE: sheet 지정 self._sheet_id(sheet_name)
                #     # "startColumnIndex": 0,
                #     # "endColumnIndex": 10
                #     # "startRowIndex": integer,
                #     # "endRowIndex": integer,
                # },
            }
        })

        body = {
            'requests': requests
        }
        response = self.service.spreadsheets().batchUpdate(
            spreadsheetId=self.id,
            body=body,
            # range = "test3"
        ).execute()

        # find_replace_response = response.get('replies')[1].get('findReplace')
        # print('{0} replacements made.'.format(
        #     find_replace_response.get('occurrencesChanged')))

        return response

    # ## NOTE: Updating Spreadsheets
    # ## https://developers.google.com/sheets/api/guides/batchupdate
    # requests = []
    # # Change the spreadsheet's title.
    # requests.append({
    #     'updateSpreadsheetProperties': {
    #         'properties': {
    #             'title': title
    #         },
    #         'fields': 'title'
    #     }
    # })
    # # Find and replace text
    # requests.append({
    #     'findReplace': {
    #         'find': find,
    #         'replacement': replacement,
    #         'allSheets': True
    #     }
    # })
    # # Add additional requests (operations) ...

    # body = {
    #     'requests': requests
    # }
    # response = service.spreadsheets().batchUpdate(
    #     spreadsheetId=spreadsheet_id,
    #     body=body).execute()
    # find_replace_response = response.get('replies')[1].get('findReplace')
    # print('{0} replacements made.'.format(
    #     find_replace_response.get('occurrencesChanged')))


##@@@ Main Function
##============================================================
if __name__ == "__main__":
    pass
    user_nick = "moonitdev"
    bot_nick = "mats"

    ##@@ NOTE: sheets
    gss = GoogleSheets(user_nick=user_nick, bot_nick=None, name="TEST__API2")
    # print(f"gss.id: {gss.id}")

    r = gss.list_of_book()
    # # r = gss.google._id_by_name("ebest_tr_list")
    # r = gss.read_sheet(sheet_name="sheet2")
    print(r)

    # r = gss._id_by_name('개인정보')
    # wb = gss.book_obj('개인정보')
    # print(f"wb: {wb.keys()}")
    # self.id = "1uauhEeAue95Y-RFOgWdFBmGyR2r8pDJpTirShwDfJrI"
    # r = gss.list_of_sheets('개인정보')
    # r = gss.add_sheet('개인정보', 'Test_Sheet')
    # r = gss.read_sheet_by_id(id, 'API')

    # r = gss.append_sheet('test_api', sheet_name="Sheet1", sheet_range=None, values=values)
    # print(r)

    ## NOTE: write(json -> sheets)
    # API_keys	125O4sk1LmV5FTtpXACRY9uaENy2QEuTY30OY6tMkpso
    # path = "configs/_client_moonitdev.json"
    # user_nick = path.rsplit("_", 1)[1].split(".", 1)[0]
    
    # with open(path, "r") as f:
    #     data = json.loads(f.read())

    # values = []
    # values.append(['user_nick'] + list(data['installed'].keys()))
    # values.append([user_nick] + [str(v) for v in list(data['installed'].values())])

    # gss.write_sheet('__API', sheet_name="google_client2", sheet_range=None, values=values)

    ## NOTE: append(json -> sheets)
    # gss.append_sheet('__API', sheet_name="google_client2", values=values)

    # NOTE: sheets -> json
    # r = gss.read_sheet('__API', sheet_name="google_client2")

    # user_nick	client_id	project_id	auth_uri	token_uri	auth_provider_x509_cert_url	client_secret	redirect_uris
    # exclusions = ['auth_provider_x509_cert_url']

    # r = [
    #     [
    #         'client_id', 'project_id', 'auth_uri', 'token_uri', 'auth_provider_x509_cert_url', 'client_secret', 'redirect_uris'
    #     ],
    #     [
    #         '981636747947-lk0gq9frkmd247kams449pgvvnvvjmqd.apps.googleusercontent.com', 'mats-299900', 'https://accounts.google.com/o/oauth2/auth', 'https://oauth2.googleapis.com/token', 'https://www.googleapis.com/oauth2/v1/certs', 'L6yKdHXtWvKW3PTukET2Vy2Y', ['urn:ietf:wg:oauth:2.0:oob', 'http://localhost']
    #     ]
    # ]
    # columns = ['user_nick', 'client_id', 'project_id', 'auth_uri']

    # # drops = r[0] - columns
    # drops = [item for item in r[0] if item not in set(columns)]
    # print(drops)
    # df = df.drop(columns=['column_nameA', 'column_nameB'])
    # df.drop(['column_nameA', 'column_nameB'], axis=1, inplace=True)

    # for i, key in enumerate(r[0]):
    #     if not key in columns:
    #         print(r[1].pop(i))

    # print(columns)
    # print(r[1])
    # data = {}
    # data['installed'] = {  ## NOTE: v가 list 이거나 dict 이면 parsing, eval(v)도 가능하나 ast.literal_eval(v)를 권장
    #     k: ast.literal_eval(v) if (v[0] == "[" and v[-1] == "]") or (v[0] == "{" and v[-1] == "}") else v 
    #     for (k, v) in zip(r[0], r[1])
    # }
    # data = {}
    # data['installed'] = _eval_read_sheet(r)
    # with open("test.json", "w") as f:
    #     json.dump(data, f, ensure_ascii=False, indent=4)


    ## NOTE: update google sheet data
    # columns = ['h4', 'h5', 'h2', 'h6', 'h9', 'h8', 'h7']
    # data = [
    #     ['v41', 'v51', 'v21', 'v61', 'v91', 'v81', 'v71'],
    #     ['v42', 'v52', 'v22', 'v62', 'v92', 'v82', 'v72'],
    #     ['v43', 'v53', 'v23', 'v63', 'v93', 'v83', 'v73'],
    # ]

    # gss.update_sheet("test3", columns, data)

    ## NOTE: update google sheet data
    # gss.replace_text("_vvv_", "-V-", sheet_name="test3")
    # gss.replace_text("-V-", "_vvv_")




importing Jupyter notebook from GoogleAPI.ipynb
importing Jupyter notebook from GoogleAuth.ipynb


NameError: name 'Google' is not defined